In [4]:
#install conda install -c conda-forge scikit-allel
import pandas as pd
import allel
import numpy as np

In [5]:
samples=['AKR_J','A_J','BALB_cJ','C3H_HeJ','CBA_J','DBA_2J','LP_J']
# tools=['sniffles']
tools=['breakdancer',
#'clever',
'delly',
'gasv',
'gridss',
#'indelminer',
#'lumpexpress',
#'mistrvar',
#'pindel',
#'platypus',
'popdel',
'smoove',
'crest',
'genomestrip',
'manta_diploidSV'
#'sniffles'
]

In [ ]:
#tools
import os
from os import path

#cov_list=['32','8','4','2','1','0.5','0.1']
#cov_list=['32','8','4','2']
cov_list=['32','16','8','4','2','1','0.5','0.1']
th_list=['0','10','100','1000','10000']
n_list=['1','2','3','4','5','6','7','8','9','10']


#n_list=['1']


df= pd.DataFrame(columns=['strain','length','flag','position','threshold'])



for s in samples:
    print ("---",s,"---")
    for t in tools:
        for th in th_list: 
            for cov in cov_list:
                for n in n_list:
            
            
                    file='../Data/raw_data/mouse/custom_vcf_'+cov+'x/'+str(th)+'t/nf_'+str(th)+'t.'+t+'.'+s+'.chr19.'+cov+'p.'+n+'_sorted.modified.vcf'
            
                    if path.exists(file):
                        callset = allel.read_vcf(file,fields='*')
                        if callset!=None:
                            df_current = pd.DataFrame({'tool': t, 'strain': s, 'length': callset['variants/SVLEN'],'flag': callset['variants/FLAG'],'position': callset['variants/POS'],'threshold': th})
                            df_current['cov']=cov
                            df_current['n']=n
                            df = pd.concat([df_current, df],ignore_index=True) 
#                         else:
#                             print("Empty File: " + file)

#                     else:
#                         print("Nonexistant File: " + file)


--- AKR_J ---


/home/vs/.local/lib/python3.5/site-packages/ipykernel_launcher.py:35: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



--- A_J ---
--- BALB_cJ ---
--- C3H_HeJ ---


In [ ]:
# df

In [ ]:
#tools
import os
from os import path

# cov_list=['32','8','4','2','1','0.5','0.1']

cov_list=['32','16','8','4','2','1','0.5','0.1']
th_list=['0','10','100','1000','10000']
#th_list=['1000']
n_list=['1','2','3','4','5','6','7','8','9','10']


#n_list=['1','2']


df_nondel= pd.DataFrame(columns=['strain','length','flag','position','threshold'])



for s in samples:
    print ("---",s,"---")
    for t in tools:
        for th in th_list: 
            for cov in cov_list:
                for n in n_list:
            
            
                    file='../Data/raw_data/mouse/custom_vcf_'+cov+'x/'+str(th)+'t/nf_'+str(th)+'t.'+t+'.'+s+'.chr19.'+cov+'p.'+n+'_sorted.modified.nondel.vcf'
            
                    if path.exists(file):
                        callset = allel.read_vcf(file,fields='*')
                        if callset!=None:
                            df_current = pd.DataFrame({'tool': t, 'strain': s, 'length': callset['variants/SVLEN'],'flag': callset['variants/FLAG'],'position': callset['variants/POS'],'threshold': th})
                            df_current['cov']=cov
                            df_current['n']=n
                            df_nondel = pd.concat([df_current, df_nondel],ignore_index=True)   
                    else:
                        print (file)

In [ ]:
df_nondel.head()

# group by TP

In [ ]:
group_data_TP=df[df['flag'] == 'TP'].groupby(['tool','threshold','strain','n','cov'],as_index=False)['flag'].count()
group_data_TP=group_data_TP.rename(columns={"flag": "nTP"})


group_data_TP[pd.isnull(group_data_TP).any(axis=1)]

In [ ]:
group_data_FP=df[df['flag'] == 'FP'].groupby(['tool','threshold','strain','n','cov'],as_index=False)['flag'].count()
group_data_FP=group_data_FP.rename(columns={"flag": "nFP"})
group_data_FP.tail()


group_data_FP[pd.isnull(group_data_FP).any(axis=1)]

In [ ]:
group_data_TN=df_nondel[df_nondel['flag'] == 'TN'].groupby(['tool','threshold','strain','n','cov'],as_index=False)['flag'].count()
group_data_TN=group_data_TN.rename(columns={"flag": "nTN"})


group_data_TN[pd.isnull(group_data_TN).any(axis=1)]


In [ ]:
#fill mssing combinations with 0s
for s in samples:
    for t in tools:
        for th in th_list:
            for cov in cov_list:
                for n in n_list:
                    if not (((group_data_TP['cov'] == cov) & (group_data_TP['n'] == n) & (group_data_TP['tool'] == t) & (group_data_TP['strain'] == s) & (group_data_TP['threshold'] == th)).any()):
                        group_data_TP = group_data_TP.append({'cov' : cov , 'n' : n , 'tool' : t , 'strain' : s,'threshold' : th,'nTP' : 0.0} , ignore_index=True)
group_data_TP[pd.isnull(group_data_TP).any(axis=1)]




In [ ]:
#fill mssing combinations with 0s
for s in samples:
    for t in tools:
        for th in th_list:
            for cov in cov_list:
                for n in n_list:
                    if not (((group_data_FP['cov'] == cov) & (group_data_FP['n'] == n) & (group_data_FP['tool'] == t) & (group_data_FP['strain'] == s) & (group_data_FP['threshold'] == th)).any()):
                        group_data_FP = group_data_FP.append({'cov' : cov , 'n' : n , 'tool' : t , 'strain' : s,'threshold' : th,'nFP' : 0.0} , ignore_index=True)
group_data_FP[pd.isnull(group_data_FP).any(axis=1)]


In [ ]:
#fill mssing combinations with 0s
for s in samples:
    for t in tools:
        for th in th_list:
            for cov in cov_list:
                for n in n_list:
                    if not (((group_data_TN['cov'] == cov) & (group_data_TN['n'] == n) & (group_data_TN['tool'] == t) & (group_data_TN['strain'] == s) & (group_data_TN['threshold'] == th)).any()):
                        group_data_TN = group_data_TN.append({'cov' : cov , 'n' : n , 'tool' : t , 'strain' : s,'threshold' : th,'nTN' : 0.0} , ignore_index=True)
group_data_TN[pd.isnull(group_data_TN).any(axis=1)]



In [ ]:
group_data_TN.head()

In [ ]:
#now both TP and FP are the size size and we can merge
df_merge=pd.merge(group_data_TP, group_data_FP,on=['tool','threshold','strain','n','cov'])
df_merge=pd.merge(df_merge, group_data_TN,on=['tool','threshold','strain','n','cov'])
df_merge[pd.isnull(df_merge).any(axis=1)]


In [ ]:
df_merge.head()

In [ ]:
#true SVs
df_current = pd.DataFrame(columns=['strain','length'])
df_true = pd.DataFrame(columns=['strain','length'])


for s in samples:
    file='../Data/gold_standard/mouse_vcf/'+s+'_reference.vcf'
    callset = allel.read_vcf(file,fields='*')
    
    df_current = pd.DataFrame({'strain': s, 'length': callset['variants/SVLEN']})
    df_true = pd.concat([df_current, df_true],ignore_index=True)
group_data_true = df_true.groupby(['strain'],as_index=False).count()
group_data_true=group_data_true.rename(columns={"length": "n_true"})
group_data_true

In [ ]:
df_merge=pd.merge(df_merge, group_data_true)
df_merge.head()

In [ ]:
df_merge.to_csv(r'FigureS10_coverage.csv')
# df_merge = pd.read_csv('Figure3_df_merge.csv')